In [6]:
import os

file_path = '../datasets/worst_errors/'

for filename in os.listdir(file_path):
    if filename.endswith('.txt'):
        with open(os.path.join(file_path, filename), 'r') as file:
            first_line = file.readline().strip()
            print(f"{filename}: {first_line}")

heap_contents_epoch_1.txt: Energy Heap: [(19.222490310668945, 'datasets/lmdb/alexandria_10_val.lmdb', 8), (32.82561492919922, 'datasets/lmdb/alexandria_10_val.lmdb', 6), (69.43992614746094, 'datasets/lmdb/alexandria_10_val.lmdb', 5)]
heap_contents_epoch_7.txt: Energy Heap: [(19.22236442565918, 'datasets/lmdb/alexandria_10_val.lmdb', 8), (32.82526397705078, 'datasets/lmdb/alexandria_10_val.lmdb', 6), (69.4393539428711, 'datasets/lmdb/alexandria_10_val.lmdb', 5)]
heap_contents_epoch_13.txt: Energy Heap: [(19.222087860107422, 'datasets/lmdb/alexandria_10_val.lmdb', 8), (32.82468032836914, 'datasets/lmdb/alexandria_10_val.lmdb', 6), (69.43816375732422, 'datasets/lmdb/alexandria_10_val.lmdb', 5)]
heap_contents_epoch_3.txt: Energy Heap: [(19.222490310668945, 'datasets/lmdb/alexandria_10_val.lmdb', 8), (32.82561492919922, 'datasets/lmdb/alexandria_10_val.lmdb', 6), (69.43992614746094, 'datasets/lmdb/alexandria_10_val.lmdb', 5)]
heap_contents_epoch_9.txt: Energy Heap: [(19.222253799438477, 'da

In [8]:
import os

# List the contents of the directory
directory_path = '../datasets/lmdb'
os.listdir(directory_path)

['alexandria_10_train.lmdb-lock',
 'alexandria_1_val.lmdb-lock',
 'alexandria_all_train.lmdb-lock',
 'alexandria_1_train.lmdb-lock',
 'alexandria_all_test.lmdb-lock',
 'alexandria_10_train.lmdb',
 'alexandria_1000_val.lmdb-lock',
 'alexandria_1_train.lmdb',
 'alexandria_all_val.lmdb-lock',
 'alexandria_all_train.lmdb',
 'real_mace',
 'alexandria_1000_test.lmdb-lock',
 'alexandria_10_test.lmdb',
 'alexandria_1000_val.lmdb',
 'alexandria_10_val.lmdb',
 'alexandria_1_test.lmdb',
 'alexandria_1000_test.lmdb',
 'alexandria_1000_train.lmdb',
 'alexandria_1000_train.lmdb-lock',
 'alexandria_10_test.lmdb-lock',
 'alexandria_1_test.lmdb-lock',
 'alexandria_1_val.lmdb',
 'alexandria_all_test.lmdb',
 'alexandria_all_val.lmdb',
 'alexandria_10_val.lmdb-lock']

In [12]:
import os

file_path = '../datasets/lmdb/alexandria_10_val.lmdb'

# Check if the file exists
file_exists = os.path.exists(file_path)

print(f"File exists: {file_exists}")

File exists: True


In [16]:
import lmdb
import pickle

file_path = '../datasets/lmdb/alexandria_10_val.lmdb'
index_to_access = 1

# Convert the index to the corresponding key
specific_key = str(index_to_access).encode('utf-8')

env = lmdb.open(file_path, subdir=False, readonly=True, lock=False)
with env.begin() as txn:
    value = txn.get(specific_key)

if value:
    entry = pickle.loads(value)
    print("Entry for index", index_to_access, ":", entry)
else:
    print("Key not found")

Entry for index 1 : Data(pos=[5, 3], cell=[1, 3, 3], atomic_numbers=[5], natoms=5, tags=[4], edge_index=[2, 96], cell_offsets=[96, 3], energy=-41.463726, forces=[5, 3], fixed=[5], sid=[1], fid=[1])


In [18]:
dir(entry)

['__annotations__',
 '__call__',
 '__cat_dim__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__inc__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_edge_attr_cls',
 '_edge_to_layout',
 '_edges_to_layout',
 '_get_edge_index',
 '_get_tensor',
 '_get_tensor_size',
 '_multi_get_tensor',
 '_put_edge_index',
 '_put_tensor',
 '_remove_edge_index',
 '_remove_tensor',
 '_store',
 '_tensor_attr_cls',
 '_to_type',
 'apply',
 'apply_',
 'batch',
 'clone',
 'coalesce',
 'concat',
 'contains_isolated_nodes',
 'contains_self_loops',
 'contiguous',
 'coo

In [20]:
entry.pos

tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 2.4384, 0.0000],
        [0.0000, 0.0000, 2.4384],
        [2.4384, 0.0000, 0.0000],
        [2.4384, 2.4384, 2.4384]])